In [1]:
# !apt-get update -qq > /dev/null
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
# !tar xf spark-2.4.8-bin-hadoop2.7.tgz
# !pip install -q findspark

The system cannot find the path specified.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-2.4.8-bin-hadoop2.7.tgz'


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/mmkshira/spark-3.2.0-bin-hadoop2.7"


import findspark
findspark.init()
findspark.find()

'/home/mmkshira/spark-3.2.0-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")
# sc = SparkContext("local", "test-app")

21/12/11 22:12:27 WARN Utils: Your hostname, scslt388 resolves to a loopback address: 127.0.1.1; using 10.0.0.26 instead (on interface wlp0s20f3)
21/12/11 22:12:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/11 22:12:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/11 22:12:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/11 22:12:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)

from pyspark.sql import SparkSession
import random

spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

## Data upload

In [7]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
data = spark.read.json("sample.jsonl").sort("year",ascending=False)
data.cache()

21/12/11 22:12:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[abstract: string, authors: array<struct<affiliation:string,affiliationid:bigint,country:string,gridid:string,id:bigint,name:string,order:bigint>>, citation_for_year: array<struct<citationcount:bigint,year:string>>, citationcount: bigint, conferenceseriesid: bigint, confname: string, confplace: string, confseries: string, confseriesname: string, countries: struct<albania:double,algeria:double,argentina:double,australia:double,austria:double,azerbaijan:double,bahrain:double,bangladesh:double,belarus:double,belgium:double,bermuda:double,bolivia:double,bosnia and herzegovina:double,botswana:double,brazil:double,british virgin islands:double,brunei:double,bulgaria:double,cambodia:double,cameroon:double,canada:double,chile:double,china:double,colombia:double,costa rica:double,croatia:double,cuba:double,cyprus:double,czechia:double,democratic republic of the congo:double,denmark:double,ecuador:double,egypt:double,estonia:double,ethiopia:double,fiji:double,finland:double,france:doubl

In [6]:
data.count()

50000

## Importing necessary packages

In [10]:
# #installing dash
# !pip install dash
# !pip install jupyter-dash
# !pip install dash_bootstrap_components

In [7]:
### importing libraries
#json
# import jsonlines
#spark
from pyspark.sql.functions import explode,col
from pyspark.sql import functions as F
from pyspark.sql.functions import concat_ws
#plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
# matplotlib and wordcloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#dash
from jupyter_dash import JupyterDash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output 
# numerical computing
import pandas as pd
import numpy as np

## Preparing the data

In [58]:
# Main Tables Used for visuals

# Main authors data
authors_data_1 = data.select("id",explode("authors").alias("authors"),"topics")

authors_data_2 = (authors_data_1.select(authors_data_1["id"].alias("paper_id"),\
    F.col("authors").getItem("name").alias("name"), \
    F.col("authors").getItem("country").alias("country"), \
    F.col("authors").getItem("id").alias("author_id"), \
    F.col("authors").getItem("affiliation").alias("affiliation"), \
    F.col("authors").getItem("affiliationid").alias("affiliationid"), \
    F.col("authors").getItem("order").alias("order"), explode("topics").alias("topic")))

authors_data_2.cache()

# Removing authors with less than 5 papers
authors_valid = authors_data_2.groupBy("author_id").agg(F.count("paper_id")).filter(col("count(paper_id)")>5)
authors_data = authors_data_2.join(authors_valid,on="author_id",how="leftsemi")

authors_data.cache()

# Topics per Author
authors_topics = authors_data.join(data.select("id","topics"),authors_data.paper_id==data.id,"left")\
            .select("author_id",explode("topics").alias("topics"))\
            .groupBy(["author_id","topics"]).count()

authors_topics.cache()

# Author Stats
# Review g-index and fix h-index
from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("author_id").orderBy(col("citationcount").desc())
authors_stats = authors_data.join(data.select("id","citationcount"),authors_data.paper_id==data.id,"left")\
                            .withColumn("h_index",F.when(F.dense_rank().over(windowSpec)<=col("citationcount"),1).otherwise(0))\
                            .groupBy("author_id")\
                            .agg(F.first("name").alias("name"),\
                                F.first("country").alias("country"),\
                                F.first("affiliation").alias("last_affiliation"),\
                                F.count("paper_id").alias("papers_count"), \
                                F.avg("order").alias("avg_order"),\
                                F.avg("citationcount").alias("avg_citations"),\
                                F.sum("citationcount").alias("total_citations"),\
                                F.sum("h_index").alias("h_index"),\
                                F.first("paper_id").alias("last_paper_id"),\
                                F.collect_list("paper_id").alias("all_paper_id"))\
                            .sort("h_index",ascending=False)
authors_stats = authors_stats.fillna({"last_affiliation":"Not Available"})
authors_stats.cache()

# Coauthors
coauthors_data = authors_data.select("paper_id","topic","author_id")\
            .join(data.select("id",explode("authors").alias("authors")),authors_data.paper_id==data.id,"left")\
            .select("paper_id", "author_id", "topic",\
                    F.col("authors").getItem("id").alias("coauthors_ids"),\
                    F.col("authors").getItem("country").alias("country"))

coauthors_data.cache()


# Map Data
map_data = px.data.gapminder().query("year==2007")[["iso_alpha","country","continent"]]


# Average Data
avg_data = authors_stats.select("papers_count","avg_order","avg_citations","total_citations","h_index")\
     .agg(F.avg("papers_count").alias("papers_count"),\
          # F.avg("avg_order").alias("avg_order"),\
          F.avg("total_citations").alias("total_citations"),\
          F.avg("avg_citations").alias("avg_citations"),\
          F.avg("h_index").alias("h_index"))\
     .toPandas()


def filtered_data(author_input):
    global filtered_stats
    filtered_stats = authors_stats.filter(col("name") == author_input.lower()).toPandas()
    return filtered_stats

21/12/11 22:57:57 WARN CacheManager: Asked to cache already cached data.
21/12/11 22:57:57 WARN CacheManager: Asked to cache already cached data.
21/12/11 22:57:57 WARN CacheManager: Asked to cache already cached data.
21/12/11 22:57:57 WARN CacheManager: Asked to cache already cached data.
21/12/11 22:57:57 WARN CacheManager: Asked to cache already cached data.


In [77]:
dt = coauthors_data.filter("author_id = 1214007197").groupBy("coauthors_ids").agg(F.count("coauthors_ids").alias("count")).sort("count", ascending = False).limit(10).select("coauthors_ids")
dt2 = dt.join(coauthors_data.filter("author_id = 1214007197").select("coauthors_ids", "author_id", "topic"),on="coauthors_ids",how="inner")

df = dt2.toPandas()#coauthors_data.filter("author_id = 1214007197").toPandas()
fig = px.parallel_categories(df, dimensions=['author_id', 'coauthors_ids', 'topic'],
                color_continuous_scale=px.colors.sequential.Inferno,
                labels={'author_id':'Author', 'coauthors_id':'Co-Author', 'topic':'Topic'})
fig.show()

## Filters

In [13]:
# filters input
# country_names = authors_data.select("country").distinct().toPandas()['country'].dropna().sort_values().tolist()
author_names_lower = authors_data.select("author_id","name").distinct().toPandas()['name'].sort_values().tolist()
author_names = [name.title() for name in author_names_lower]
# author_names

## MAIN DASHBOARD

In [53]:
app = JupyterDash(external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div([ 
          dbc.Container([
             dbc.Row([
                  dbc.Col(
                          html.H1(children = "Author Statistics",
                                  style = {'color':'#fcc200', 'align':'center'},
                                  className="text-center mt-4 mb-5"), 
                      align="center", className="mt4")       
             ]),
             dbc.Row([
                   dbc.Col(
                      html.Div(children = [
                          html.H5("Author", style = {'width': '50vw','display': 'inline-block'}),
                          dcc.Dropdown(id = "author-select",
                                    options = [{'label': i, 'value': i} for i in author_names],
                                    value = "Alexander Wong",
                                    style = {'width': '50vw','display': 'inline-block', 'color':'black', "size":14}
                                    )],  className="text-center mt-4 mb-5"),
                      align="center",
                      width = 12)], 
                   justify = "center"),
             dbc.Row([
                      dbc.Col([
                       dbc.Card(
                           dbc.CardBody(
                               [html.H6(children='Total Papers Published',
                                       style={
                                            'textAlign': 'center',
                                            'color': 'white'}
                                        ),
                              html.H4(
                                  id = "papers-metric",
                                  style={
                                          'textAlign': 'center',
                                          'color': 'blue',
                                          'fontSize': 40,
                                          'height':"3vw"},
                                  )]
                             )       
                       ) 
                      ]),
                      dbc.Col([
                       dbc.Card(
                           dbc.CardBody(
                               [html.H6(children='Last Affiliation',
                                       style={
                                            'textAlign': 'center',
                                            'color': 'white'}
                                        ),
                              html.H4(
                                  id = "affiliation-metric",
                                  style={
                                          'textAlign': 'center',
                                          'color': 'orange',
                                          'fontSize': 25,
                                          'height':"3vw"}
                                  )]
                             )       
                       ) 
                      ]),
                      dbc.Col([
                       dbc.Card(
                           dbc.CardBody(
                               [html.H6(children='Citations Received',
                                       style={
                                            'textAlign': 'center',
                                            'color': 'white'}
                                        ),
                              html.H4(
                                  id = "citations-metric",
                                  style={
                                          'textAlign': 'center',
                                          'color': 'red',
                                          'fontSize': 40,
                                          'height':"3vw"}
                                  )]
                             )       
                       ) 
                      ]),
                      dbc.Col([
                       dbc.Card(
                           dbc.CardBody(
                               [html.H6(children='H-Index',
                                       style={
                                            'textAlign': 'center',
                                            'color': 'white'}
                                        ),
                              html.H4(
                                  id = "hindex-metric",
                                  style={
                                          'textAlign': 'center',
                                          'color': 'yellow',
                                          'fontSize': 40,
                                          'height':"3vw"}
                                  )]
                             )       
                       ) 
                      ]),
                      dbc.Col([
                       dbc.Card(
                           dbc.CardBody(
                               [html.H6(children='Favourite Topic',
                                       style={
                                            'textAlign': 'center',
                                            'color': 'white'}
                                        ),
                              html.H4(
                                  id = "topics-metric",
                                  style={
                                          'textAlign': 'center',
                                          'color': 'pink',
                                          'fontSize': 25,
                                          'height':"3vw"}
                                  )]
                             )       
                       ) 
                      ]),
             ]),
            #  dbc.Row([
            #           dbc.Col(html.Div(id="text-info"), width = 8)
            #           ]),
             html.Br(),
             dbc.Row([
                    dbc.Col([
                            dcc.Graph(id="papers-trend")], 
                            align="left", width = 4, 
#                             style={"height": "50vw", "width": "40vw"}
                           ),
                    dbc.Col([
                        dcc.Graph(id="avg-bar-chart")],
#                                       style={"height": "50vw", "width": "40vw"})], 
                            align="center")
             ]),
             html.Br(),
             dbc.Row([
                 dbc.Col([
                      dcc.Graph(id="authors-table")],
                      align="center"
                         )
    
          ])
    ])
])

## card 1
@app.callback(Output(component_id="papers-metric", component_property="children"),
               Input(component_id="author-select", component_property="value")
               )

def papers_card(author_input):
    filtered_stats = filtered_data(author_input)
    filtered_stats = authors_stats.filter(col("name") == author_input.lower()).toPandas()
    return filtered_stats["papers_count"][0]

## card 2
@app.callback(Output(component_id="affiliation-metric", component_property="children"),
               Input(component_id="author-select", component_property="value")
               )

def affiliation_card(author_input):
    filtered_stats = authors_stats.filter(col("name") == author_input.lower()).toPandas()
    return filtered_stats["last_affiliation"][0]

## card 3
@app.callback(Output(component_id="citations-metric", component_property="children"),
               Input(component_id="author-select", component_property="value")
               )

def citations_card(author_input):
    filtered_stats = authors_stats.filter(col("name") == author_input.lower()).toPandas()
    return filtered_stats["total_citations"][0]

## card 4
@app.callback(Output(component_id="hindex-metric", component_property="children"),
               Input(component_id="author-select", component_property="value")
               )

def hindex_card(author_input):
    filtered_stats = authors_stats.filter(col("name") == author_input.lower()).toPandas()
    return filtered_stats["h_index"][0]

## card 5
@app.callback(Output(component_id="topics-metric", component_property="children"),
               Input(component_id="author-select", component_property="value")
               )

def topics_card(author_input):
    
    author_id_selected = filtered_stats["author_id"].unique()
    filtered_topics = authors_topics.filter(col("author_id") == author_id_selected.tolist()[0]).distinct().sort("count", ascending=False).toPandas()
    return filtered_topics["topics"][1].title()


# line chart
@app.callback(Output(component_id="papers-trend", component_property="figure"),
               Input(component_id="author-select", component_property="value")
               )

def papers_trend_fig(author_input):
    summary = filtered_stats
      # summary = authors_stats.filter(col("name")==author_input).toPandas()
    df = (data.filter(col("id").isin(summary["all_paper_id"].iloc[0]))
                .groupBy("year").count()
                .toPandas())
    df['year'] = pd.DatetimeIndex(df['year']).year
    years = pd.DataFrame({'year':range(df.year.min(), df.year.max())})
    df = df.merge(years, how='outer').fillna(0).sort_values("year")
    df = df.rename(columns={"year": "Year", "count": "Number of Papers"})

    fig = px.line(df, x="Year", y="Number of Papers", range_x = [min(df.Year), max(df.Year)])
    fig.layout.template = "plotly_dark"
    title = "Trend Line for papers published"
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',paper_bgcolor='rgba(0, 0, 0, 0)',
                     width = 500, height = 500, 
                     title=title, title_x=0.5,
                     margin=dict(l=80, r=50, t=80, b=20))
    return fig


# bar chart          
@app.callback(Output(component_id="avg-bar-chart", component_property="figure"),
               Input(component_id="author-select", component_property="value")
               )

def avg_bar_authors(author_input):
    summary = filtered_stats
    fig = make_subplots(rows=1, cols=5)
    i = 1;flag = True
    for label in avg_data.columns: 
        fig_1 = go.Figure(data=[
                  go.Bar(name='Average', x=[label], y=[avg_data[label].iloc[0]],
                        marker_color='#ffdf00',showlegend=flag),
                  go.Bar(name='Selected Author', x=[label], y=[summary[label].iloc[0]],
                        marker_color='blue',showlegend=flag)
                  ])
        fig.add_trace(fig_1["data"][0],row=1,col=i)
        fig.add_trace(fig_1["data"][1],row=1,col=i)  
        i=i+1
        flag = False

#     fig.update_layout(height=600, width=1000)
    fig.layout.template = "plotly_dark"
    title = f"Author's Stats Compared to Average"
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',paper_bgcolor='rgba(0, 0, 0, 0)', 
                      width = 800, height = 500, 
                      title=title, title_x=0.5,
                      margin=dict(l=80, r=10, t=80, b=20))
    return fig


# table
@app.callback(Output(component_id="authors-table", component_property="figure"),
               Input(component_id="author-select", component_property="value")
               )

def table_author_papers(author_input):
    summary = filtered_stats
  # summary = authors_stats.filter(col("author_id")==author_id).toPandas()
    df = data.filter(col("id").isin(summary["all_paper_id"].iloc[0]))\
           .select("papertitle","citationcount","topics","year","doi","urls")\
           .toPandas()
    fig = go.Figure(data=[go.Table(
        columnorder = [1,2,3,4,5,6],
        columnwidth = [150,80,150,80,80,150],
      header=dict(values=["Title","Citation Count","Topics","Year","Doi","Url"],
                  fill_color='black',
                  align='left',
                  font=dict(color="white", size = 14)),
      cells=dict(values=[df.papertitle,df.citationcount,df.topics.apply(lambda x: ', '.join(map(str, x))),df.year,df.doi,df.urls],
                fill_color='#fafad2', font=dict(color='black', size=14),
                align='left'))
    ])
    fig.layout.template = "plotly_dark"
    title = f"Top papers by {author_input}"
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',paper_bgcolor='rgba(0, 0, 0, 0)', title=title, title_x=0.5,
                      margin=dict(l=20, r=20, t=80, b=20),
                      height = 500)
    return fig

app.run_server(mode="external",port="8082")

Dash app running on http://127.0.0.1:8082/
